In [2]:
import wikipedia
import pandas as pd
import numpy as np
import re


Make function for wiki pages. 

In [98]:
import wikipedia
import requests
import urllib.parse
from bs4 import BeautifulSoup
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm
URL = "https://randomincategory.toolforge.org/Random_page_in_category?"
URL += f"category={urllib.parse.quote('famous fictional characters')}"
URL += f"&category2={urllib.parse.quote('historical figures')}"
URL += f"&category3={urllib.parse.quote('slogans')}"
URL += f"&category4={urllib.parse.quote('catchphrases')}"
URL += f"&category5={urllib.parse.quote('authors')}"
URL += f"&category6={urllib.parse.quote('actors')}"
URL += f"&category7={urllib.parse.quote('british authors')}"
URL += f"&category8={urllib.parse.quote('american authors')}"
URL += f"&category9={urllib.parse.quote('musicians')}"
URL += f"&category10={urllib.parse.quote('songs')}"
URL += f"&category11={urllib.parse.quote('books')}"
URL += f"&category12={urllib.parse.quote('movies')}"
URL += "&server=en.wikipedia.org&cmnamespace=&cmtype=page&returntype="


replacements = {
    r'\bsex\b': 'affection',
    r'\bporn\w*\b': 'adult entertainment',
    r'fuck': 'have intimate relations with',
    r'fucking': 'romancing, physically',
    r'\bfuck\w*\b': ' (bleep) ',
    r'\bass\b': 'dump truck',
    r'\bshit\w*\b': 'poop',
    r'damn\w*\b': 'darn',
    r'god-damn\w*\b|goddamn\w*\b': 'super darn',
    r'\bass\b|\wasse*': 'rear end',
    r'cock': 'rooster',
    r'small dick energy':'little man syndrome',
    r'\bdick\w*\b': ' johnson ',
    r'ppl':'people',
    '\bwat\b':'what',
    r'a black': 'an african-american',
    r'black person':'dark skinned person',
    r'\bpee\b' : 'urinate',
    r'\bpiss\b': 'urinate',
    r'\b(peeing|pissing)': 'urinating',
    r'white person':'caucasian',
    r'\bwhore\w*\b': 'prostitute',
    r'nigg*': 'racial slur',
    '\r\n':' ',
    r'\bslut\b': ' loose person ',
    r'\bblowjob\b': 'random gift',
    r'racist': 'ignorant',
    'racism':'fear of people that look different',
    r'faggot': 'gay person',
    r'\bfag\w*\b':'gay person',
    r'boob|boob*|breast': 'lady pillows',
    r'\bbitch*\b': 'mean woman',
    r'bastard*': 'illegitimate child',
    r'hoes?': 'chicks',
    r'breast*|jugs': 'chest',
    r'\bcunt*\b': 'comtemptible person',
    r'\bpuss\w*\b': 'vagina',
    r'\wdick*': 'penis',
    r'naked': 'disrobed',
    r'\bnud\w*\b': 'unclothed',
    r'\nmasterbate\w*\b': 'self-gratified',
    r'\bmasturbating\w*\b': 'gratifying themselves',
    r'\b\w{4}ilf\b': 'person id like to get to know',
    r'mastu\w*\b': 'self-gratification',
    'god':'deity',
    'jesus':'religious figure',
    'christ':'religious figure',
    'bible':'religious text',
    'church':'place of worship',
    'religion':'set of beliefs',
    r'\bpray\w*\b':'communicate with a deity',
    'prayer':'a conversation with a deity',
    'faith':'belief',
    ' lord ':'captain',
    ' allah ':'a diety',
    'gay':'homosexual',
    r' rapist ': ' intense toucher ',
    r' rape ': ' unwelcomed tickling ', # eek I know... 
    r'pedo\w*\b': 'seventies mustached van driver',
    'yahwey':'a deity',
    'yeshua':'a religious figure',
    ' sexual':' reproductive',
    'douche':'chad',
    ' sex ':' private adult time ',
    'milf':"mother I would like to take on a date",
    ' butt ':"dumptruck/tailgate"
}

def replacer_censor(definition,phrase,replacements_dict):
    # Iterate over the keys in the replacements dictionary
    for pattern in replacements_dict:
        # Use re.sub to replace the occurrences of the pattern with its corresponding value in the phrase and definition strings
        phrase = re.sub(pattern, replacements_dict[pattern], phrase)
        definition = re.sub(pattern, replacements_dict[pattern], definition)
    return phrase, definition


def unpack_definitions(phrase,definition):
    # remove the brackets and clean up the definitions
    # with regex
    definition = definition.replace("[","")
    definition = definition.replace("]","")
    definition = definition.replace("'","")
    definition = definition.replace('"',"")
    definition = definition.replace("(","")
    definition = definition.replace(")","")

    # remove double spaces
    definition = definition.replace('  ',' ')
    # also remove any .. by replacing them with a single .
    definition = definition.replace('..','.')
    # remove all non-ascii characters
    definition = re.sub(r'[^\x00-\x7f]',r'', definition)
    # remove any words that are not in the english dictionary
    #english_words = set(w.lower() for w in nltk.corpus.words.words())
    #definition = re.sub(r'\b\w+\b', lambda m: m.group(0) if m.group(0) in english_words else '', definition)
    # remove extra spaces
    #definition = re.sub(' +', ' ', definition)
    
    phrase, definition = replacer_censor(definition,phrase,replacements)
    

    return phrase, definition
# limit to 1 request per second
@sleep_and_retry
def get_random_wiki_entry():

    #print("getting random wiki entry from url: ",URL)
    # use requests to get the page, then pass the redirected page url to wikipedia library.
    req_url = requests.get(URL).url # this is a random page from the category
    # using wikipedia library
    page = wikipedia.page(req_url) # this is a random page from the category
    title = page.title
    summary = page.summary
    #categories = page.categories
    related = page.links
    # create a dictionary of the entry
    random_wiki_entry_dict = {
        "title": title,
        "summary": unpack_definitions(title,summary),
        "related": related,
    }

    return random_wiki_entry_dict

# make a card from the random wiki entry
def create_ppn_card():
    # create a person/place/thing card
    # get a random wikipedia entry
    random_wiki_entry_dict = get_random_wiki_entry()
    # get the summary of the entry
    random_wiki_entry_summary = random_wiki_entry_dict["summary"]
    # get the title of the entry
    random_wiki_entry_title = random_wiki_entry_dict["title"]
    # get the categories of the entry
    random_wiki_entry_related = random_wiki_entry_dict["related"]

    # create a dictionary of the entry
    random_wiki_entry_dict = {
        "title": random_wiki_entry_title,
        "summary": random_wiki_entry_summary,
        "related": len(random_wiki_entry_related)
    }

    return random_wiki_entry_dict

# create a deck of fifty cards from the random wiki entries (non-repeating) and return the deck as a list of dictionaries

def make_one_card():
    while True:
        try:
            card = create_ppn_card() # create a card
        except Exception as e:
            continue
        else:
            if len(card["summary"][1]) > 10 and 'Wiki' not in card["title"]: # if the card is valid
                break
            else:
                continue
    return card

def create_ppn_deck(card_count=50):
    # create a deck of card_count cards
    ppn_deck = []
    for i in tqdm(range(card_count)):
        ppn_deck.append(make_one_card())
    return ppn_deck


In [99]:
card_deck = create_ppn_deck(50) # create a deck of 50 cards
# show the deck

  8%|▊         | 4/50 [00:19<03:40,  4.80s/it]


KeyboardInterrupt: 

# Summarization

In [ ]:

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
# import LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
def summarize_text(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return summary

text = "This is some text that I want to summarize. It can be as long or as short as needed. The summary will be a few sentences long."
num_sentences = 2
summary = summarize_text(text, num_sentences)

for sentence in summary:
    print(sentence)


This is some text that I want to summarize.
The summary will be a few sentences long.


# go through the deck and summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
sentence_count = 2
for card in card_deck:
    summary = summarize_text(card["summary"][1], int(sentence_count))
    summary_short = ""
    for sentence in summary:
        summary_short += str(sentence)
    card["summary_short"] = summary_short

# show the deck
print(f'I have created a deck of {len(card_deck)} cards')


import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

# These are the available summarization methods in sumy
summarizers = [
    "LsaSummarizer", 
    "LexRankSummarizer", 
    "KLSummarizer", 
    "TextRankSummarizer", 
    "SumBasicSummarizer",
    "LuhnSummarizer",
]

# This function demonstrates how to use each summarization method
def demonstrate_summarizers(text, language="english"):
    parser = PlaintextParser.from_string(text, Tokenizer(language))
    for summarizer in summarizers:
        summarizer = eval(summarizer)() # create an instance of the summarizer
        summary = summarizer(parser.document, 3) # summarize the document with 3 sentences
        print("====== Example: ", summarizer," ======")

        for sentence in summary:
            print(sentence)
        print()
    

text = "Reddit ; stylized in all lowercase as reddit is an American social news aggregation, content rating, and discussion website. Registered users commonly referred to as Redditors submit content to the site such as links, text posts, images, and videos, which are then voted up or down by other members. Posts are organized by subject into user-created boards called communities or subreddits. Submissions with more upvotes appear towards the top of their subreddit and, if they receive enough upvotes, ultimately on the sites front page. Reddit administrators moderate the communities. Moderation is also conducted by community-specific moderators, who are not Reddit employees.As of March 2022, Reddit ranks as the 9th-most-visited website in the world and 6th most-visited website in the U.S., according to Semrush. About 4249.3% of its user base comes from the United States, followed by the United Kingdom at 7.98.2% and Canada at 5.27.8%. "
demonstrate_summarizers(text)


!pip install pyenchant

In [102]:
# clean the text of the summary field of each card by removing all special characters, replacing newlines with spaces, numbers with their word equivalent, and removing all words that are not in the english dictionary.

import re
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
# import LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# create a list of english words
english_words = words.words()

# go through the deck and summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
sentence_count = 2
for card in card_deck:
    summary = summarize_text(card["summary"][1], int(sentence_count))
    summary_short = ""
    for sentence in summary:
        summary_short += str(sentence)
    card["summary_short"] = summary_short

# show the deck
print(f'I have created a deck of {len(card_deck)} cards')


correct_cards = 0
while len(card_deck) < 50:
    # create a deck of fifty cards from the random wiki entries (non-repeating) and return the deck as a list of dictionaries
    card_deck = create_ppn_deck(50) # create a deck of 50 cards
    
    # go through the deck and summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
    sentence_count = 2
    for card in card_deck:
        summary = summarize_text(card["summary"][1], int(sentence_count))
        summary_short = ""
        for sentence in summary:
            summary_short += str(sentence)
        card["summary_short"] = summary_short

    # remove any cards with words in the title that are in the banned words list
    banned_words = ['List']

    for card in card_deck:
        for word in banned_words:
            if word in card["title"]:
                card_deck.remove(card)

    # clean the text of the summary field of each card by removing all special characters, replacing newlines with spaces, numbers with their word equivalent, and removing all words that are not in the english dictionary.
    for card in card_deck:
        card["summary_clean"] = re.sub(r"[^a-zA-Z0-9]+", ' ', card["summary_short"])
        card["summary_clean"] = card["summary_clean"].replace("\n", " ")

    # if a card has more than 1/4 of its words in the english dictionary, then it is valid, else drop the card
    for card in card_deck:
        card["summary_clean"] = word_tokenize(card["summary_clean"])
        card["summary_clean"] = [word for word in card["summary_clean"] if word in english_words]
        if len(card["summary_clean"]) < len(card["summary_short"])/4:
            card_deck.remove(card)

    # use unpack_definitions on the summary_clean field to remove unusual bad words
    for card in card_deck:
        card["summary_clean"] = unpack_definitions(card['title'],card["summary_clean"])


    # show the deck
    print(f'I have created a deck of {len(card_deck)} cards')


    # truncate all page summaries to be no more than 1200 characters, ending at the nearest sentence boundary. This is to prevent the summary from being too long for the flashcard.
    for card in card_deck:
        summary = card["summary_short"]
        if len(summary) > 1200:
            summary = summary[:1200]
            summary = summary[:summary.rfind('.')+1]
            card["summary_short"] = summary

    # drop all cards with a summary that is less than 100 characters long
    card_deck = [card for card in card_deck if len(card["summary_short"]) > 100]

    # use LexRankSummarizer to summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
    for card in card_deck:
        if card['status'] == 'good':
            continue
        summarizer = LexRankSummarizer() # create an instance of the summarizer
        parser = PlaintextParser.from_string(card["summary"][1], Tokenizer("english"))
        summary = summarizer(parser.document, 3) # summarize the document with 3 sentences
        summary_short = ""
        for sentence in summary:
            summary_short += str(sentence)
        card["summary_short"] = summary_short
        card['status'] = 'good'


I have created a deck of 50 cards


In [103]:
print(f'I have created a deck of {len(card_deck)} cards')
for card in card_deck:
    print(card["title"])
    print(card["summary_short"])
    print(card["related"])
    print()

I have created a deck of 50 cards
List of TCP and UDP port numbers
Similarly, many of the official assignments refer to protocols that were never or are no longer in common use.This article lists port numbers and their associated protocols that have experienced significant uptake.
1000

Jewish Community of Vienna
The Jewish Community of Vienna Israelitische Kultusgemeinde Wien or IKG is the body that represents Viennas Orthodox Jewish community.Throughout history, it has represented almost all of Austrias Jews, whose numbers are sufficient to form communities in only a few other cities in Austria.
57

Timeline of Russian interference in the 2016 United States elections
This is a timeline of events related to alleged Russian interference in the 2016 United States elections.It includes events described in investigations into suspected inappropriate links between associates of Donald Trump and Russian officials until July 2016, with July 2016 through election day November 8, 2016, followi

In [105]:
# save the deck to a json file by combining the card dictionaries into a json file format.

import json

with open('ppn_deck.json', 'w') as outfile:
    json.dump(card_deck, outfile, indent=4)
